In [85]:
# import data
import pandas as pd
df = pd.read_parquet('./data/catB_train.parquet')

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,...,recency_giclaim,giclaim_cnt_success,recency_giclaim_success,giclaim_cnt_unsuccess,recency_giclaim_unsuccess,flg_gi_claim_29d435_ever,flg_gi_claim_058815_ever,flg_gi_claim_42e115_ever,flg_gi_claim_856320_ever,f_purchase_lh
19550,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,1974-05-09,Female,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
4600,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,1979-11-11,Male,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
13337,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,1976-01-28,Male,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
15074,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,1976-03-19,Female,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
19724,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,1995-07-31,Female,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN


### General Client Information  
• clntnum: Unique identifier for the client.  
• race_desc: Description of the client's race.  
• ctrycode_desc: Country code indicating the client's location.  
• clttype: Customer status.  
• **stat_flag**: Flag indicating ACTIVE, LAPSED or MATURED. E.g. if there’s at least one inforce policy, then the flag would be ACTIVE. If all of the client’s policies are all lapsed, then it is LAPSED.  
• min_occ_date: Date of the client's first interaction or policy purchase with the company.  
• **cltdob_fix**: Fixed or corrected date of birth of the client.  
• **cltsex_fix**: Fixed or corrected gender of the client.  

In [86]:
num_rows, num_columns = df.shape
df.head()

# Print the results
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

Number of rows: 17992
Number of columns: 304


In [3]:
# count classes of categorical variables
set(df['stat_flag'].values.tolist())

{'ACTIVE', 'LAPSED', 'MATURED'}

In [4]:
# check presence of missing value
sum(pd.isnull(df['stat_flag']))

0

In [100]:
print(df['hh_20'])

19550     144
4600      153
13337      62
15074       1
19724     114
         ... 
11284      85
11964       0
5390       43
860        72
15795    None
Name: hh_20, Length: 17992, dtype: object


In [92]:
set(df['annual_income_est'].values.tolist())

{'A.ABOVE200K', 'B.100K-200K', 'C.60K-100K', 'D.30K-60K', 'E.BELOW30K', None}

In [5]:
set(df['clttype'].values.tolist())

{'C', 'G', 'P'}

### Client Risk and Status Indicators  
• flg_substandard: Flag for substandard risk clients.  
• flg_is_borderline_standard: Flag for borderline standard risk clients.  
• flg_is_revised_term: Flag if customer ever has revised terms.  
• flg_is_rental_flat: Indicates if the client lives in a rental flat.  
• flg_has_health_claim: Flag for clients with health insurance claims.  
• flg_has_life_claim: Flag for clients with life insurance claims.  
• flg_gi_claim: Flag for general insurance claims.  
• flg_is_proposal: Indicates if there is a policy in proposal for client.  
• flg_with_preauthorisation: Flag for clients with preauthorized transactions or policies.  
• flg_is_returned_mail: Flag for returned mail instances.  

### Demographic and Household Information  
• is_housewife_retiree, is_sg_pr, is_class_1_2: Flags indicating specific demographics like occupation, residency status, etc.  
• is_dependent_in_at_least_1_policy: Indicates if the client is a dependent in at least one policy.  
• hh_20, pop_20, hh_size, hh_size_est: Metrics related to household size and population.  
• annual_income_est: Estimated annual income of the client, in buckets.  

In [90]:
def analyze_binary_variable(df, column_name):
    print(f"Data type for {column_name}: {df[column_name].dtype}")
    # Print count of missing data
    print(f"Count of missing data for {column_name}: {sum(pd.isnull(df[column_name]))}")

    # Filter rows without missing values
    df_no_missing = df[df[column_name].notnull()]

    # Display count of each value
    count_true = (df[column_name] == 1).sum()
    count_false = (df[column_name] == 0).sum()
    print(f"Count of {column_name} being True (1): {count_true}")
    print(f"Count of {column_name} being False (0): {count_false}")

    # Display unique values in the filtered DataFrame
    unique_values = set(df_no_missing[column_name].values.tolist())
    print(f"Unique values for {column_name}: {unique_values}")
analyze_binary_variable(df, 'is_housewife_retiree')
analyze_binary_variable(df, 'is_sg_pr')
analyze_binary_variable(df, 'is_class_1_2')
analyze_binary_variable(df, 'is_dependent_in_at_least_1_policy')

def analyze_numerical_variable(df, column_name):
    print(f"Data type for {column_name}: {df[column_name].dtype}")
    # Print count of missing data
    print(f"Count of missing data for {column_name}: {sum(pd.isnull(df[column_name]))}")

    # Filter rows without missing values
    df_no_missing = df[df[column_name].notnull()]

    result = {}

    # Calculate mean
    result['mean'] = df_no_missing[column_name].mean()

    # Calculate median
    result['median'] = df_no_missing[column_name].median()

    # Calculate mode (using scipy.stats.mode as pandas mode can return multiple modes)
    result['mode'] = stats.mode(df_no_missing[column_name]).mode[0]

    # Calculate variance using pandas
    result['variance'] = df_no_missing[column_name].var()

    # Detect outliers using the IQR method
    Q1 = df_no_missing[column_name].quantile(0.25)
    Q3 = df_no_missing[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df_no_missing[(df_no_missing[column_name] < lower_bound) | (df_no_missing[column_name] > upper_bound)]
    result['outlier# Print the resultss'] = outliers
    
    # print(f"Mean: {result_dict['mean']}")
    #print(f"Median: {result_dict['median']}")
    #print(f"Mode: {result_dict['mode']}")
    #print(f"Variance: {result_dict['variance']}")
    #print("\nOutliers:")
    #print(result_dict['outliers'])

    return result

Data type for is_housewife_retiree: float64
Count of missing data for is_housewife_retiree: 1014
Count of is_housewife_retiree being True (1): 54
Count of is_housewife_retiree being False (0): 16924
Unique values for is_housewife_retiree: {0.0, 1.0}
Data type for is_sg_pr: float64
Count of missing data for is_sg_pr: 1014
Count of is_sg_pr being True (1): 15593
Count of is_sg_pr being False (0): 1385
Unique values for is_sg_pr: {0.0, 1.0}
Data type for is_class_1_2: float64
Count of missing data for is_class_1_2: 1014
Count of is_class_1_2 being True (1): 9051
Count of is_class_1_2 being False (0): 7927
Unique values for is_class_1_2: {0.0, 1.0}
Data type for is_dependent_in_at_least_1_policy: float64
Count of missing data for is_dependent_in_at_least_1_policy: 1014
Count of is_dependent_in_at_least_1_policy being True (1): 0
Count of is_dependent_in_at_least_1_policy being False (0): 16978
Unique values for is_dependent_in_at_least_1_policy: {0.0}


In [70]:
# is_housewife_retiree

print(f"Count of missing data: {sum(pd.isnull(df['is_housewife_retiree']))}") 
# Display rows without missing values
df_no_missing = df[df['is_housewife_retiree'].notnull()]

# Now, df_no_missing contains only the rows without missing values
result = df_no_missing.loc[:, ['is_housewife_retiree']]
count_no_missing = (df['is_housewife_retiree'] == 0).sum()
count_housewife = (df['is_housewife_retiree'] == 1).sum()
print(f"Count of not retired housewife: {count_no_missing}")
print(f"Count of retired housewife: {count_housewife}")
df['is_housewife_retiree'].info()

Count of missing data: 1014
Count of not retired housewife: 16924
Count of retired housewife: 54
<class 'pandas.core.series.Series'>
Index: 17992 entries, 19550 to 15795
Series name: is_housewife_retiree
Non-Null Count  Dtype  
--------------  -----  
16978 non-null  float64
dtypes: float64(1)
memory usage: 281.1 KB


In [71]:
# is_sg_pr
print(f"Count of missing data: {sum(pd.isnull(df['is_sg_pr']))}") 
# Display rows without missing values
df_no_missing_pr = df[df['is_sg_pr'].notnull()]

# Now, df_no_missing contains only the rows without missing values
result = df_no_missing_pr.loc[:, ['is_sg_pr']].sum()
count_not_pr = (df['is_sg_pr'] == 0).sum()
count_pr = (df['is_sg_pr'] == 1).sum()
print(f"Count of not_pr: {count_not_pr}")
print(f"Count of pr: {count_pr}")
df['is_sg_pr'].info()

Count of missing data: 1014
Count of not_pr: 1385
Count of pr: 15593
<class 'pandas.core.series.Series'>
Index: 17992 entries, 19550 to 15795
Series name: is_sg_pr
Non-Null Count  Dtype  
--------------  -----  
16978 non-null  float64
dtypes: float64(1)
memory usage: 281.1 KB


In [66]:
# is_class_1_2
print(f"Count of missing data: {sum(pd.isnull(df['is_class_1_2']))}") 

# Display rows without missing values
df_no_missing_class = df[df['is_class_1_2'].notnull()]

# Now, df_no_missing contains only the rows without missing values
result = df_no_missing_class.loc[:, ['is_class_1_2']]
# set(df_no_missing_class['is_class_1_2'].values.tolist())
count_class_2 = (df['is_class_1_2'] == 1).sum()
count_class_1 = (df['is_class_1_2'] == 0).sum()
print(f"Count of class 2: {count_class_2}")
print(f"Count of class 1: {count_class_1}")
df['is_class_1_2'].info()

Count of missing data: 1014
Count of class 2: 9051
Count of class 1: 7927
<class 'pandas.core.series.Series'>
Index: 17992 entries, 19550 to 15795
Series name: is_class_1_2
Non-Null Count  Dtype  
--------------  -----  
16978 non-null  float64
dtypes: float64(1)
memory usage: 281.1 KB


In [64]:
# is_dependent_in_at_least_1_policy
print(f"Count of missing data: {sum(pd.isnull(df['is_dependent_in_at_least_1_policy']))}") 

# Display rows without missing values
df_no_missing_dependency = df[df['is_dependent_in_at_least_1_policy'].notnull()]

# Now, df_no_missing contains only the rows without missing values
result = df_no_missing_dependency.loc[:, ['is_dependent_in_at_least_1_policy']]
# set(df_no_missing_class['is_class_1_2'].values.tolist())
count_dependent = (df['is_dependent_in_at_least_1_policy'] == 1).sum()
count_independent = (df['is_dependent_in_at_least_1_policy'] == 0).sum()
print(f"Count of dependent: {count_dependent}")
print(f"Count of independent: {count_independent}")
df['is_dependent_in_at_least_1_policy'].info()
set(df_no_missing_dependency['is_dependent_in_at_least_1_policy'].values.tolist())

Count of missing data: 1014
Count of dependent: 0
Count of independent: 16978
<class 'pandas.core.series.Series'>
Index: 17992 entries, 19550 to 15795
Series name: is_dependent_in_at_least_1_policy
Non-Null Count  Dtype  
--------------  -----  
16978 non-null  float64
dtypes: float64(1)
memory usage: 281.1 KB


{0.0}

In [94]:
# annual_income_est
print(f"Count of missing data: {sum(pd.isnull(df['annual_income_est']))}") 

# Display rows without missing values
df_no_missing_income = df[df['annual_income_est'].notnull()]

# Now, df_no_missing contains only the rows without missing values
result = df_no_missing_income.loc[:, ['annual_income_est']]
# set(df_no_missing_class['is_class_1_2'].values.tolist())
count_A = (df_no_missing_income['annual_income_est'].str.split('.').str[0] == 'A').sum()
count_B = (df_no_missing_income['annual_income_est'].str.split('.').str[0] == 'B').sum()
count_C = (df_no_missing_income['annual_income_est'].str.split('.').str[0] == 'C').sum()
count_D = (df_no_missing_income['annual_income_est'].str.split('.').str[0] == 'D').sum()
count_E = (df_no_missing_income['annual_income_est'].str.split('.').str[0] == 'E').sum()
print(f"Count of ABOVE200K: {count_A}")
print(f"Count of 100K-200K: {count_B}")
print(f"Count of 60K-100K: {count_C}")
print(f"Count of 30K-60K: {count_D}")
print(f"Count of BELOW30K: {count_E}")
df['annual_income_est'].info()
set(df_no_missing_dependency['annual_income_est'].values.tolist())

Count of missing data: 2809
Count of ABOVE200K: 2097
Count of 100K-200K: 725
Count of 60K-100K: 2679
Count of 30K-60K: 1911
Count of BELOW30K: 7771
<class 'pandas.core.series.Series'>
Index: 17992 entries, 19550 to 15795
Series name: annual_income_est
Non-Null Count  Dtype 
--------------  ----- 
15183 non-null  object
dtypes: object(1)
memory usage: 281.1+ KB


{'A.ABOVE200K', 'B.100K-200K', 'C.60K-100K', 'D.30K-60K', 'E.BELOW30K'}

In [97]:
# f_purchase_lh
df_no_missing_purchase = df[df['f_purchase_lh'].notnull()]
result = df_no_missing_purchase.loc[:, ['annual_income_est']]
count_A = (df_no_missing_purchase['annual_income_est'].str.split('.').str[0] == 'A').sum()
count_B = (df_no_missing_purchase['annual_income_est'].str.split('.').str[0] == 'B').sum()
count_C = (df_no_missing_purchase['annual_income_est'].str.split('.').str[0] == 'C').sum()
count_D = (df_no_missing_purchase['annual_income_est'].str.split('.').str[0] == 'D').sum()
count_E = (df_no_missing_purchase['annual_income_est'].str.split('.').str[0] == 'E').sum()
print(f"Count of ABOVE200K: {count_A}")
print(f"Count of 100K-200K: {count_B}")
print(f"Count of 60K-100K: {count_C}")
print(f"Count of 30K-60K: {count_D}")
print(f"Count of BELOW30K: {count_E}")

Count of ABOVE200K: 141
Count of 100K-200K: 43
Count of 60K-100K: 140
Count of 30K-60K: 80
Count of BELOW30K: 287


### Policy and Claim History  
• n_months_last_bought_products, flg_latest_being_lapse, flg_latest_being_cancel, recency_lapse, recency_cancel: Metrics related to the recency of policy purchases, lapses, and cancellations.  
• tot_inforce_pols, tot_cancel_pols: Total number of in-force and canceled policies.  
• f_ever_declined_la: Flag for clients has ever been declined policies.  

### Anonymized Insurance Product Metrics (APE, Sum Insured, Prepaid Premiums)  
• ape_, sumins_, prempaid_* (e.g., ape_gi_42e115, sumins_ltc_1280bf, prempaid_grp_6fc3e6): Metrics for various anonymized insurance products, likely representing different types of policies like general insurance, long-term care, group policies, etc. The suffixes (like 42e115, 1280bf) are unique identifiers for the specific insurance products. ‘ape’ stands for Annual Premium Equivalent, 'sumins' for sum insured, ‘prempaid’ stands for premium customers will pay from product inception to product maturity.  

### Other Flags and Metrics  
• f_elx, f_mindef_mha, f_retail: Flags possibly related to client's association with specific programs or sectors.  
• flg_affconnect_*, affcon_visit_days, n_months_since_visit_affcon: Metrics related to client’s activity in affinity connect.  
• clmcon_visit_days, recency_clmcon, recency_clmcon_regis: Metrics related to client’s activity in claim connect.  
• hlthclaim_amt, giclaim_amt, recency_hlthclaim, recency_giclaim, hlthclaim_cnt_success, giclaim_cnt_success: Health and general insurance claim-related metrics.  
• flg_hlthclaim_, flg_gi_claim_ (e.g., flg_hlthclaim_839f8a_ever, flg_gi_claim_29d435_ever): Flags for specific types of health and general insurance claims, with anonymized identifiers.  

### Purchase and Lapse Metrics for Specific Products  
• f_ever_bought_, n_months_last_bought_, lapse_ape_, n_months_since_lapse_ (e.g., f_ever_bought_839f8a, n_months_last_bought_grp_6fc3e6, lapse_ape_ltc_1280bf, n_months_since_lapse_inv_dcd836): Flags and metrics indicating purchase history, lapses, and time since last interaction for various anonymized insurance products..